# Diff in Diff Regression by Clarissa

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/20_analysis/big_merge.csv")
df_clean = df.dropna(axis=0).copy()
df.sample(3)

,Unnamed: 0,Unnamed: 0.1,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment,year
3326,382,382,female,Okara,rural,348,270.0,0.775862,2012
5717,389,389,male,Orakzai,rural,332,193.0,0.581325,2019
952,236,236,female,Chitral,rural,289,168.0,0.581315,2006


In [2]:
# Packages
import numpy as np
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm

## Aggregated

In [3]:
# Treatment and pre-post variables

df_clean.loc[(df_clean["year"] >= 2010), "post_2009"] = 1
df_clean.loc[(df_clean["year"] < 2007), "post_2009"] = 0

# Divide into urban/rural and rural+female

df_urban = df_clean.loc[df_clean["region"] == "urban"].copy()
df_rural = df_clean.loc[df_clean["region"] == "rural"].copy()
df_female_rural = df_clean.loc[(df_clean["sex"] == "female") & (df_clean["region"] == "rural")].copy()

## Cities controlled by terrorist in 2009 vs not
(only measuring differences in women)

In [4]:
taliabn_dominance = [
    "South Waziristan",
    "North Waziristan",
    "Orakzai",
    "Kurram",
    "Khyber",
    "Mohmand",
    "Bajur",
    "Darra Adamkhel",
    #"Swat",
    "Upper Dir",
    "Lower Dir",
    "Bannu",
    "Lakki Marwat",
    "Tank",
    "Peshawar",
    "Dera Ismail Khan",
    "Mardan",
    "Charsadda",
    "Kohat",
]

In [5]:
# checking if the names are spelled the same way as we have in our dataset
for i in taliabn_dominance:
    print('{}: {}'.format(i,(df_female_rural["subprovince"].isin(taliabn_dominance)==True).any()))

South Waziristan: True
North Waziristan: True
Orakzai: True
Kurram: True
Khyber: True
Mohmand: True
Bajur: True
Darra Adamkhel: True
Upper Dir: True
Lower Dir: True
Bannu: True
Lakki Marwat: True
Tank: True
Peshawar: True
Dera Ismail Khan: True
Mardan: True
Charsadda: True
Kohat: True


In [6]:
# DEFINE: Treatment = in taliban dominated district

df_female_rural["Treated"] = 0
df_female_rural.loc[(df_female_rural["subprovince"].isin(taliabn_dominance)), "Treated"] = 1

# check
Table1 = pd.crosstab(df_female_rural['year'],df_female_rural['Treated'])
Table1

Treated,0,1
year,,
2004,88,9
2005,72,9
2006,92,9
2007,71,9
2008,101,9
2010,105,9
2011,81,9
2012,105,9
2013,107,9


In [7]:
df_female_rural[df_female_rural["subprovince"].isin(taliabn_dominance)]['subprovince'].unique()

array(['Bannu', 'Charsadda', 'Kohat', 'Lakki Marwat', 'Lower Dir',
       'Mardan', 'Peshawar', 'Tank', 'Upper Dir', 'Bajur', 'Khyber',
       'Kurram', 'Mohmand', 'North Waziristan', 'Orakzai',
       'South Waziristan'], dtype=object)

In [8]:
print(Table1.to_latex(caption=('full_caption', 'short_caption')))

\begin{table}
\centering
\caption[short_caption]{full_caption}
\begin{tabular}{lrr}
\toprule
Treated &    0 &   1 \\
year &      &     \\
\midrule
2004 &   88 &   9 \\
2005 &   72 &   9 \\
2006 &   92 &   9 \\
2007 &   71 &   9 \\
2008 &  101 &   9 \\
2010 &  105 &   9 \\
2011 &   81 &   9 \\
2012 &  105 &   9 \\
2013 &  107 &   9 \\
2014 &   97 &   9 \\
2015 &  108 &   9 \\
2018 &   16 &  16 \\
2019 &  105 &  16 \\
\bottomrule
\end{tabular}
\end{table}



In [9]:
from linearmodels import PanelOLS
df_for_panelols = df_female_rural.set_index(["subprovince", "year"])

mod = PanelOLS.from_formula(
    "rate_enrollment ~ Treated * post_2009 + EntityEffects + TimeEffects",
    data=df_for_panelols,
    drop_absorbed=True,
).fit()
mod.summary

/Users/clarissaache/opt/miniconda3/lib/python3.9/site-packages/linearmodels/shared/exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
/Users/clarissaache/opt/miniconda3/lib/python3.9/site-packages/linearmodels/panel/model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Treated, post_2009

  warnings.warn(


Dep. Variable:,rate_enrollment,R-squared:,0.0034
Estimator:,PanelOLS,R-squared (Between):,0.0125
No. Observations:,1089,R-squared (Within):,0.0096
Date:,"Mon, Apr 25 2022",R-squared (Overall):,0.0101
Time:,18:46:34,Log-likelihood,1011.0
Cov. Estimator:,Unadjusted,,
,,F-statistic:,3.2631
Entities:,130,P-value,0.0712
Avg Obs:,8.3769,Distribution:,"F(1,948)"
Min Obs:,1.0000,,
Max Obs:,19.000,F-statistic (robust):,3.2631


In [10]:
print(mod.summary.as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}     &  rate\_enrollment  & \textbf{  R-squared:         }   &      0.0034      \\
\textbf{Estimator:}         &      PanelOLS      & \textbf{  R-squared (Between):}  &      0.0125      \\
\textbf{No. Observations:}  &        1089        & \textbf{  R-squared (Within):}   &      0.0096      \\
\textbf{Date:}              &  Mon, Apr 25 2022  & \textbf{  R-squared (Overall):}  &      0.0101      \\
\textbf{Time:}              &      18:46:34      & \textbf{  Log-likelihood     }   &      1011.0      \\
\textbf{Cov. Estimator:}    &     Unadjusted     & \textbf{                     }   &                  \\
\textbf{}                   &                    & \textbf{  F-statistic:       }   &      3.2631      \\
\textbf{Entities:}          &        130         & \textbf{  P-value            }   &      0.0712      \\
\textbf{Avg Obs:}           &       8.3769       & \textbf{  Distribution:      }   &     F(1,948)     \\


In [11]:
# Taliban proximity

mod_proximity = smf.ols('rate_enrollment ~ C(post_2009) * C(Treated)', df_female_rural).fit()
mod_proximity.get_robustcov_results(cov_type="HC3").summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        rate_enrollment   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     21.01
Date:                Mon, 25 Apr 2022   Prob (F-statistic):           3.10e-13
Time:                        18:46:34   Log-Likelihood:                 28.141
No. Observations:                1089   AIC:                            -48.28
Df Residuals:                    1085   BIC:                            -28.31
Df Model:                           3                                         
Covariance Type:                  HC3                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               0.4774      0.014     33.237      0.000       0.449       0.506
C(post_2009)[T.1.0]                     0.0966      0.017      5.673      0.000       0.063       0.130
C(Treated)[T.1]                        -0.0679      0.027     -2.475      0.013      -0.122      -0.014
C(post_2009)[T.1.0]:C(Treated)[T.1]     0.0276      0.036      0.762      0.446      -0.043       0.099
==============================================================================
Omnibus:                      196.629   Durbin-Watson:                   1.677
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               43.913
Skew:                          -0.098   Prob(JB):                     2.91e-10
Kurtosis:                       2.036   Cond. No.                         12.7
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [12]:
print(mod_proximity.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}                       & rate\_enrollment & \textbf{  R-squared:         } &     0.036   \\
\textbf{Model:}                               &       OLS        & \textbf{  Adj. R-squared:    } &     0.033   \\
\textbf{Method:}                              &  Least Squares   & \textbf{  F-statistic:       } &     13.54   \\
\textbf{Date:}                                & Mon, 25 Apr 2022 & \textbf{  Prob (F-statistic):} &  1.12e-08   \\
\textbf{Time:}                                &     18:46:34     & \textbf{  Log-Likelihood:    } &    28.141   \\
\textbf{No. Observations:}                    &        1089      & \textbf{  AIC:               } &    -48.28   \\
\textbf{Df Residuals:}                        &        1085      & \textbf{  BIC:               } &    -28.31   \\
\textbf{Df Model:}                            &           3      & \textbf{                     } &             \\
\textbf{Covariance Type:}         